In [1]:
import sys
import os
from tqdm import tqdm_notebook
import pandas
from pandas.tseries.holiday import USFederalHolidayCalendar
from statsmodels.tsa.arima_model import ARIMA
import sklearn.model_selection

os.chdir('/home/chlang/Projects/prognosec')
sys.path.append('/home/chlang/Projects/prognosec/prognosec')

In [2]:
gdp = pandas.read_csv('data/USGDP/usgdp.csv', sep=',')
gdp = gdp.rename({'level-current': 'gdp-current', 'level-chained': 'gdp-chained',
                  'change-current': 'gdp-change-current',
                  'change-chained': 'gdp-change-chained'}, axis=1)
gdp['date'] = pandas.to_datetime(gdp['date'], format='%Y-%m-%d')
gdp = gdp.set_index('date').asfreq('QS', 'ffill')

cpiai = pandas.read_csv('data/USCPIAI/uscpiai.csv', sep=',')
cpiai = cpiai.rename({'Date': 'date', 'Index': 'cpi', 'Inflation': 'inflation'}, axis=1)
cpiai['date'] = pandas.to_datetime(cpiai['date'], format='%Y-%m-%d')
cpiai = cpiai.set_index('date').asfreq('MS', 'ffill')

In [3]:
gdp.index

DatetimeIndex(['1947-04-01', '1947-07-01', '1947-10-01', '1948-01-01',
               '1948-04-01', '1948-07-01', '1948-10-01', '1949-01-01',
               '1949-04-01', '1949-07-01',
               ...
               '2015-01-01', '2015-04-01', '2015-07-01', '2015-10-01',
               '2016-01-01', '2016-04-01', '2016-07-01', '2016-10-01',
               '2017-01-01', '2017-04-01'],
              dtype='datetime64[ns]', name='date', length=281, freq='QS-JAN')

## Testing method for upsampling/downsampling

**upsampling** means to resample low frequency (e.g. monthly) to high frequency (e.g. daily). For example, taking monthly CPI and giving it the same value for the days in that month

**downsampling** means to resample high frequency (e.g. daily) to low frequency (e.g. monthly). Some form of aggregation is needed e.g. mean, sum

### Attempt upsampling (e.g. quarterly -> monthly)